In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline


covid_data = pd.read_csv('data/covid_data.csv')

vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]

# Чтобы убраль из таблицы колонку province мы суммируем остальные колонки  
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

# Преобразуем дату
covid_data['date'] = pd.to_datetime(covid_data['date'])

# Создаем новый признак "Чисто заболевших на данный момент" active 
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

# Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого 
# отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам 
# и рассчитаем разницу между «вчера и сегодня» с помощью метода diff()
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])


# Exercise 3.1
# За какой период представлены данные в таблице covid_data? В качестве ответа 
# введите даты в формате datetime (без указания времени)

'''display(
    covid_data['date'].min(),
    covid_data['date'].max()
    )'''


# Exercise 3.2 
# За какой период представлены данные в таблице vaccinations_data? 
# В качестве ответа введите даты в формате datetime без указания времени.

'''display(
    vaccinations_data['date'].min(),
    vaccinations_data['date'].max()
)'''


# Exercise 3.3
# Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, 
# вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи 
# из таблицы covid_data и из её пересечения с vaccinations_data, но не более. 
# Результат объединения занесите в переменную covid_df.

covid_df = covid_data.merge(
    vaccinations_data,
    on=['date', 'country'],
    how='left'
    )

covid_df['death_rate'] = covid_df['deaths']/covid_df['confirmed'] * 100
covid_df['recover_rate'] = covid_df['recovered']/covid_df['confirmed'] *100


# Exercise 3.4
# Какова максимальная летальность в США (United States) за весь период? 
# Ответ округлите до второго знака после запятой.

covid_df[covid_df['country'] == 'United States']['death_rate'].max().round(2)


# Exercise 3.5
# Чему равен средний процент выздоровевших в России (Russia)?
'''
covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean().round(2)


fig = plt.figure(figsize=(8,4))
axis = fig.add_axes([0,0,1,1])
ru_death = covid_df[covid_df['country'] == 'Russia'][['daily_deaths', 'date']]

axis.scatter(
    x = ru_death['date'],
    y = ru_death['daily_deaths'],
    s=100,
    marker='^',
    c='blue'
)
'''

# Exercise 6.4
countries = ['Russia', 'Australia', 'Germany', 'Canada', 'United Kingdom']
croped_covid_df = covid_df[covid_df['country'].isin(countries)]

populations = pd.DataFrame([
    ['Canada', 37664517],
    ['Germany', 83721496],
    ['Russia', 145975300],
    ['Australia', 25726900],
    ['United Kingdom', 67802690]
    ],
    columns=['country', 'population']
)
croped_covid_df = croped_covid_df.merge(populations, on=['country'])
croped_covid_df['daily_confirmed_per_hundred'] = croped_covid_df['daily_confirmed'] / croped_covid_df['population'] * 100
croped_covid_df.head()

croped_covid_df['confirmed_per_hundred'] = croped_covid_df['confirmed']/croped_covid_df['population'] * 100

'''pivot = croped_covid_df.pivot_table(
    values='confirmed_per_hundred',
    index='country',
    columns='date'
)
pivot.columns = pivot.columns.astype('string')
fig = plt.figure(figsize=(10, 7))
heatmap = sns.heatmap(pivot, cbar='blue')
heatmap.set_title('Тепловая карта заболеваемости', fontsize=16)'''
# display(heatmap)

'''fig = plt.figure(figsize=(10,5))
boxplot = sns.boxplot(
    data=croped_covid_df,
    y='country',
    x='recover_rate',
    orient='h'
)'''

line_data = covid_df.groupby('country', as_index=False).sum()
fig = px.line(
    data_frame=line_data, #датафрейм
    x='country', #ось абсцисс
    y=['confirmed', 'recovered', 'deaths', 'active'], #ось ординат
    height=500, #высота
    width=1000, #ширина
    title='Confirmed, Recovered, Deaths, Active cases over Time' #заголовок
)
fig.show()

C:\Users\sasa\AppData\Local\Temp\ipykernel_12672\1161043795.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  line_data = covid_df.groupby('country', as_index=False).sum()
